In [1]:
import boto3
import httpx
import json
import os

### Set variables and helper functions

In [2]:
def checkFilePath(file_path):
    result = s3.list_objects(Bucket=bucket, Prefix=file_path)
    exists = True if 'Contents' in result else False
    if exists:
        print('PATH EXISTS')
        return result['Contents']
    return exists

In [3]:
user = os.getenv('CHE_WORKSPACE_NAMESPACE')
titiler_endpoint = "https://titiler.maap-project.org"  # MAAP titiler endpoint
bucket = "maap-ops-workspace"
rescale = "0,50"

#### Option to query possible projections supported by titiler service

In [25]:
tileMatrixSets = httpx.get(f"{titiler_endpoint}/tileMatrixSets").json()
print("Supported TMS:")
for tms in tileMatrixSets["tileMatrixSets"]:
    print("-", tms["id"])

Supported TMS:
- CanadianNAD83_LCC
- EuropeanETRS89_LAEAQuad
- LINZAntarticaMapTilegrid
- NZTM2000
- NZTM2000Quad
- UPSAntarcticWGS84Quad
- UPSArcticWGS84Quad
- UTM31WGS84Quad
- WGS1984Quad
- WebMercatorQuad
- WorldCRS84Quad
- WorldMercatorWGS84Quad


### Right-click on the file and select option to Copy Path

In [4]:
path = input("Path to raster in bucket:")

Path to raster in bucket: my-private-bucket/Mabounie_AGB_50m.tif


In [5]:
s3 = boto3.client('s3')
file_name = path.split('/', 1)[-1]
if 'shared-buckets' in path:
    file_path = f'shared/{file_name}'
if 'my-private-bucket' in path:
    file_path = f'{user}/{file_name}'
if 'my-public-bucket' in path:
    file_path = f'shared/{user}/{file_name}'
print(checkFilePath(file_path))

PATH EXISTS
[{'Key': 'emmalu/Mabounie_AGB_50m.tif', 'LastModified': datetime.datetime(2023, 8, 2, 20, 2, 26, tzinfo=tzutc()), 'ETag': '"346716ac16211e54eb6b82c17189e7ca"', 'Size': 228859, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'MSFC-IMPACT-MAAP-Ops-root', 'ID': '801c37e81ec7d7b327915c96502ec5f346f48f2cdc819da9284110dbc39b64e7'}}, {'Key': 'emmalu/Mabounie_AGB_50m.tif.aux.xml', 'LastModified': datetime.datetime(2023, 8, 24, 20, 8, 6, tzinfo=tzutc()), 'ETag': '"1979e84bcfb72e246198ca3870213823"', 'Size': 439, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'MSFC-IMPACT-MAAP-Ops-root', 'ID': '801c37e81ec7d7b327915c96502ec5f346f48f2cdc819da9284110dbc39b64e7'}}]


In [7]:
url = f"s3://maap-ops-workspace/{file_path}"

#### If Path exists, continue...

### Open raster from url

In [8]:
import rioxarray as rxr

raster = rxr.open_rasterio(url, masked=True)
raster

<xarray.DataArray (band: 1, y: 242, x: 302)>
[73084 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6.65e+05 6.65e+05 6.651e+05 ... 6.8e+05 6.8e+05
  * y            (y) float64 9.921e+06 9.921e+06 ... 9.909e+06 9.909e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        597.03997802734
    STATISTICS_MEAN:           308.99170271785
    STATISTICS_MINIMUM:        0.00017995000234805
    STATISTICS_STDDEV:         80.274742739481
    STATISTICS_VALID_PERCENT:  97.1
    scale_factor:              1.0
    add_offset:                0.0

#### Project to default map projection

In [9]:
crs = raster.rio.crs
print("The CRS of this dataset is:", crs)
crs_number = crs.to_epsg()
if crs_number != 3857:
    raster = raster.rio.reproject("EPSG:3857")
    crs = raster.rio.crs
    print("The NEW CRS of this dataset is:", crs)

The CRS of this dataset is: EPSG:32732
The NEW CRS of this dataset is: EPSG:3857


#### Get raster info (bounds, zoom, data type)

In [10]:
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

# print(json.dumps(r, indent=4))

bounds = r.get("bounds")
minzoom = r.get("minzoom")
zoom = minzoom + 1 if minzoom == 0 else minzoom
bands = r.get("band_metadata")

print("Bounds:", bounds)
print("Zoom:", zoom)
print("Data type:", r.get("dtype"))
print("Bands:", bands)

Bounds: [10.48231191030377, -0.8234798450869963, 10.618013429251134, -0.7139983750010857]
Zoom: 12
Data type: float32
Bands: [['b1', {}]]


#### Calculate raster center for map placement

In [11]:
from shapely.geometry import box

polygon = box(*bounds)
center = (polygon.centroid.y, polygon.centroid.x)
print("Center:", center)

Center: (-0.768739110044041, 10.55016266977745)


#### Get value statistics for rescaling

In [12]:
r = httpx.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url,
    }
).json()

# print(json.dumps(r, indent=4))
band = r.get("b1")
if band:
    minv, maxv = band.get("min"), band.get("max")
    print("minv:", minv, "maxv:", maxv)
    rescale = f"{minv}, {maxv}"

minv: 0.00017995000234805048 maxv: 597.0399780273438


### Display local raster

#### Create TileLayer

In [13]:
from ipyleaflet import TileLayer

colormap = "gist_heat"

r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "rescale": rescale,
        "colormap_name": colormap,
    }
).json()

tile_url = r["tiles"][0]
custom_layer = TileLayer(url=tile_url, show_loading=True, visible=True, opacity=.5)

In [14]:
import stac_ipyleaflet

m = stac_ipyleaflet.StacIpyleaflet(zoom=zoom, center=center)
m.add_layer(custom_layer)
m

Output()

StacIpyleaflet(center=[-0.768739110044041, 10.55016266977745], controls=(ZoomControl(options=['position', 'zoo…

#### If adding layer after map was created, use the fit_bounds method to navigate to it
`m.fit_bounds(bounds)`

#### Option to remove custom layer
`m.remove_layer(custom_layer)`